In [43]:
from cloudsafe import *
database = 'here-api/main.db'
conn = create_connection(database)
departures = select_all_departures(conn)
stations = select_all_stations(conn)
bus_stops = select_all_bus_stops()

In [47]:
import folium
from folium.plugins import HeatMap
coords = stations.iloc[0].coordinates_here.split(',')
m = folium.Map([coords[0], coords[1]], zoom_start=10)

def get_coords(row):
    if row['coordinates_here']:
        coords = row['coordinates_here'].split(',') 
        return coords[0], coords[1]
    return 0, 0

stations[['longitude', 'latitude']] = stations.apply(get_coords, axis=1, result_type="expand")

for index, row in stations.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=15,
                        popup=row['name_here'],
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m)



In [50]:
# convert to (n, 2) nd-array format for heatmap
stationArr = stations[['latitude', 'longitude']].to_numpy()

# plot heatmap
m.add_children(HeatMap(stationArr, radius=15))
m

AttributeError: module 'folium' has no attribute 'plugins'